In [1]:
! pip install kaggle


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
! mkdir ~/.kaggle


mkdir: /Users/haifaalmkhdoub/.kaggle: File exists


In [4]:
# the path to the json file , please upload a json file in yourgoogle drive
!cp  ./kaggle.json  ~/.kaggle


In [ ]:
! kaggle datasets download  ismailnasri20/driver-drowsiness-dataset-ddd


Dataset URL: https://www.kaggle.com/datasets/ismailnasri20/driver-drowsiness-dataset-ddd
License(s): unknown
... resuming from 444596224 bytes (2327385414 bytes left) ...
 34%|█████████████▏                         | 891M/2.58G [00:22<01:33, 19.7MB/s]

In [8]:
! unzip  ./driver-drowsiness-dataset-ddd.zip

Archive:  ./driver-drowsiness-dataset-ddd.zip
replace Driver Drowsiness Dataset (DDD)/Drowsy/A0001.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [9]:
# Imports here
import numpy as np
import torch
from torch import nn
from torch import optim
from torchvision import  transforms, models , datasets
from torch.utils.data import DataLoader , Dataset
from tqdm import tqdm  # For progress bars
from collections import OrderedDict
from PIL import Image
import json
from collections import OrderedDict
from sklearn.model_selection import KFold
import os
import numpy as np
from collections import defaultdict

In [10]:
#for my specific device's GPU  adjust for other hardware as needed.

print(torch.__version__)
import torch

if torch.backends.mps.is_available():
    print("GPU is available.")
    device = torch.device('mps')
else:
    print("GPU is not available.")
    device = torch.device('cpu')






2.5.0
GPU is available.


In [20]:
#!pip install split-folders


In [11]:
# create a class for our data set to use dataloders
class DriverDrowsinessDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label


In [22]:
 # delete later
#import splitfolders

#data_dir = '/content/Driver Drowsiness Dataset (DDD)'
#output_dir = '/content/splitted_Data'
#splitfolders.ratio(data_dir, output=output_dir, seed=1337, ratio=(0.8, 0.15, 0.05))

In [23]:
# delete later
#train_dir = "/content/splitted_Data/train"
#test_dir = "/content/splitted_Data/train"
#valid_dir = "/content/splitted_Data/val"



In [13]:


base_folder = './Driver Drowsiness Dataset (DDD)'
drowsy_folder = os.path.join(base_folder, 'Drowsy')
non_drowsy_folder = os.path.join(base_folder, 'Non Drowsy')

video_groups = defaultdict(list)

folders = [drowsy_folder, non_drowsy_folder]
labels_dict = {}

for folder in folders:
    label = 0 if 'Non Drowsy' in folder else 1
    for img_name in os.listdir(folder):
        video_id = img_name[0].lower()  # Convert starting letter to lowercase
        # in the dataset all frames belonging to the same video start  with the same letter except for z which has 3 diffrent videos  za zb zc
        if video_id == 'z' :
            video_id = img_name[0:2].lower()
        video_groups[video_id].append(os.path.join(folder, img_name))
        labels_dict[os.path.join(folder, img_name)] = label

# sepreate test set
video_group_keys = list(video_groups.keys())
np.random.shuffle(video_group_keys)
test_size = int(0.2 * len(video_group_keys))
test_groups = video_group_keys[:test_size]
train_val_groups = video_group_keys[test_size:]

# Create file paths and labels for test set
test_files = [file for group in test_groups for file in video_groups[group]]
test_labels = [labels_dict[file] for file in test_files]


k_folds = 5
folds = [[] for _ in range(k_folds)]

for idx, video_id in enumerate(train_val_groups):
    folds[idx % k_folds].extend(video_groups[video_id])


for fold_idx, fold_files in enumerate(folds, 1):
    print(f"Fold {fold_idx}")
    # print groups in each fold
    starting_letters = []
    for file in fold_files:
      base_name = os.path.basename(file).lower()
      if base_name[0]=='z':
        starting_letters.append(base_name[0:2])
      else:
        starting_letters.append(base_name[0])

    unique_starting_letters = sorted(set(starting_letters))
    print(f"Starting Letters: {unique_starting_letters}")
    print(f"Total files in fold: {len(fold_files)}")
    print("-" * 30)



Fold 1
Starting Letters: ['g', 'n', 's', 't', 'za']
Total files in fold: 6273
------------------------------
Fold 2
Starting Letters: ['d', 'h', 'o', 'v', 'zc']
Total files in fold: 8320
------------------------------
Fold 3
Starting Letters: ['e', 'k', 'm', 'x', 'zb']
Total files in fold: 10060
------------------------------
Fold 4
Starting Letters: ['a', 'b', 'i', 'q']
Total files in fold: 6610
------------------------------
Fold 5
Starting Letters: ['c', 'u', 'w', 'y']
Total files in fold: 5932
------------------------------


In [15]:
data_transforms = {
    'train': transforms.Compose([
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
                ]),
    'valid': transforms.Compose([transforms.Resize(255),
                               transforms.CenterCrop(224),
                                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
                              ]),
    'test': transforms.Compose([transforms.Resize(255),
                               transforms.CenterCrop(224),
                                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
                              ])

}
'''
# TODO: Load the datasets with ImageFolder
image_datasets = {
    'train': datasets.ImageFolder(train_dir,transform = data_transforms['train']),
    'valid': datasets.ImageFolder(valid_dir,transform = data_transforms['valid']),
    'test':datasets.ImageFolder(test_dir,transform= data_transforms['test'])

}

# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders ={'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True),
    'valid': DataLoader(image_datasets['valid'], batch_size=32, shuffle=True),
    'test': DataLoader(image_datasets['test'], batch_size=32, shuffle=True)
}
'''

"\n# TODO: Load the datasets with ImageFolder\nimage_datasets = {\n    'train': datasets.ImageFolder(train_dir,transform = data_transforms['train']),\n    'valid': datasets.ImageFolder(valid_dir,transform = data_transforms['valid']),\n    'test':datasets.ImageFolder(test_dir,transform= data_transforms['test'])\n\n}\n\n# TODO: Using the image datasets and the trainforms, define the dataloaders\ndataloaders ={'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True),\n    'valid': DataLoader(image_datasets['valid'], batch_size=32, shuffle=True),\n    'test': DataLoader(image_datasets['test'], batch_size=32, shuffle=True)\n}\n"

In [16]:
model = models.resnet50(pretrained=True)
print(model)



/opt/anaconda3/envs/flowers_env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/flowers_env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [17]:
# freeze paramters
for param in model.parameters():
    param.requires_grad = False

In [18]:
# this just a base code we can change the number of nodes and layers
fc = nn.Sequential(OrderedDict([
    ('fc1',nn.Linear(2048,512,bias=True)),
    ('relu' ,nn.ReLU()),
    ('dropout',nn.Dropout(0.2)),
    ('f2', nn.Linear(512,512,bias=True)),
    ('dropout',nn.Dropout(0.2)),
    ('f3', nn.Linear(512,1,bias=True)),]))
model.fc = fc

In [19]:

def train_model(model, trainloader, validloader, criterion, optimizer, num_epochs=20, print_every=2, device='cpu'):

    for e in range(num_epochs):

        # Training Phase
        train_accuracy = 0
        running_loss = 0
        model.train()

        for images, labels in tqdm(trainloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            log_ps = model(images)  # Forward pass (raw logits)
            loss = criterion(log_ps, labels.view(-1, 1).float())  # Calculate loss
            loss.backward()  # Backpropagation
            optimizer.step()

            running_loss += loss.item()

            # Calculate accuracy for training
            with torch.no_grad(): 
                ps = torch.sigmoid(log_ps)  
                top_class = ps >= 0.5  # Threshold of 0.5 for binary classification (1 or 0)
                equals = top_class == labels.view(*top_class.shape)  #
                train_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

        # Print average losses and validation accuracy at specified intervals
        if (e + 1) % print_every == 0:
            test_loss = 0
            valid_accuracy = 0
            model.eval()  # Set to evaluation mode
            with torch.no_grad():  # Disable gradient computation during validation
                for inputs, labels in tqdm(validloader):
                    inputs, labels = inputs.to(device), labels.to(device)
                    log_ps = model(inputs)  # Forward pass (raw logits)
                    batch_loss = criterion(log_ps, labels.view(-1, 1).float())
                    test_loss += batch_loss.item()

                    # Calculate accuracy for validation
                    ps = torch.sigmoid(log_ps)  # Apply sigmoid for thresholding
                    top_class = ps >= 0.5  # Use threshold of 0.5 for classification
                    equals = top_class == labels.view(*top_class.shape)
                    valid_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

            # Print training and validation statistics
            print(f"Epoch {e+1}/{num_epochs}.. "
                  f"Train loss: {running_loss/len(trainloader):.3f}.. "
                  f"Training accuracy: {train_accuracy/len(trainloader):.3f}.. "
                  f"Validation loss: {test_loss/len(validloader):.3f}.. "
                  f"Validation accuracy: {valid_accuracy/len(validloader):.3f}")

            # Reset variables for next epoch
            running_loss = 0
            train_accuracy = 0

            model.train()  # Switch back to training mode

    return model, test_loss , valid_accuracy



True

In [ ]:
all_fold_val_losses = []
all_fold_val_accuracies = []

# Cross-validation loop
for fold_idx in range(k_folds):
    print
    val_files = folds[fold_idx]
    train_files = []
    for idx  in range(len(folds)):
      if idx != fold_idx:
          for file in folds[idx]:
            train_files.append(file)

    val_labels = [labels_dict[file] for file in val_files]
    train_labels = [labels_dict[file] for file in train_files]

    train_dataset = DriverDrowsinessDataset(train_files, train_labels, transform = data_transforms['train'])
    val_dataset = DriverDrowsinessDataset(val_files, val_labels, transform = data_transforms['valid'])
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,drop_last = True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True,drop_last = True)
    criterion =  nn.BCEWithLogitsLoss()
    model.to(device)
    optimizer = optim.Adam(fc.parameters(), lr=0.003)
    trained_model, valid_loss, valid_accuracy  = train_model(model, train_loader ,val_loader, criterion, optimizer, num_epochs=20 ,device = device)
    all_fold_val_losses.append(valid_loss)
    all_fold_val_accuracies.append(valid_accuracy)







In [40]:
def test_model(model, testloader, criterion):
    model.eval()  # Set model to evaluation mode
    test_loss = 0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():  # Turn off gradients for testing
        for images, labels in tqdm(testloader):
            images, labels = images.to(device), labels.to(device)  # Move to device
            log_ps = model(images)  # Forward pass
            loss = criterion(log_ps, labels.view(-1, 1).float())
            test_loss += loss.item()
            ps = torch.sigmoid(log_ps)  # Apply sigmoid for thresholding
            top_class = ps >= 0.5  # Use threshold of 0.5 for classification
            equals = top_class == labels.view(*top_class.shape)
            test_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

    test_accuracy = total_correct / total_samples
    print(f"Test Accuracy: {test_accuracy:.3f}")

In [ ]:


def test_model(model, test_loader, criterion, device):
    model.eval() 
    test_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    
    with torch.no_grad():  # Disable gradient calculation for testing
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels.view(-1, 1).float())
            test_loss += loss.item() * images.size(0)  # Accumulate test loss
            
            # Convert outputs to binary predictions
            predictions = (torch.sigmoid(outputs) >= 0.5).float()
            correct_predictions += (predictions == labels).sum().item()
            total_samples += labels.size(0)
    

    average_test_loss = test_loss / total_samples
    accuracy = correct_predictions / total_samples
    
    print(f"Test Loss: {average_test_loss:.4f}")
    print(f"Test Accuracy: {accuracy * 100:.2f}%")
    
    return average_test_loss, accuracy


test_dataset = DriverDrowsinessDataset(test_files, test_labels, transform=data_transforms['test'])
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Run the test function
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
test_loss, test_accuracy = test_model(model, test_loader, criterion, device)


In [ ]:
# code copied from another project i worked on still need updating to fit our project

In [ ]:
#save  trained model   to a file to use later
model.class_to_idx = image_datasets['train'].class_to_idx

checkpoint = {
    'epoch': 30,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'class_to_idx': model.class_to_idx
}

torch.save(checkpoint, 'model_checkpoint.pth')


In [ ]:
#  function that loads a checkpoint and rebuilds the model
checkpoint = torch.load('./model_checkpoint.pth')
model = models.densenet121(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

Classifer= nn.Sequential(OrderedDict([
    ('fc1',nn.Linear(100,512,bias=True)),
    ('relu' ,nn.ReLU()),
    ('dropout',nn.Dropout(0.2)),
    ('f2', nn.Linear(512,102)),
    ('output',nn.LogSoftmax(dim=1)) ]))
model.fc = fc
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
model.class_to_idx = checkpoint['class_to_idx']

